# Install Requirements

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# GPU

In [2]:
#pip install tensorflow[and-cuda]
#pip install tensorrt

# CPU

In [3]:
#pip install tensorflow

In [1]:
import pandas as pd
import numpy as np
from helper import create_xy
#import tensorrt
import tensorflow as tf
import keras
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout
from keras.models import Sequential
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt

2024-07-16 15:29:10.134101: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-16 15:29:10.134128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-16 15:29:10.201400: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-16 15:29:10.252796: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-07-16 15:29:31.600025: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-16 15:29:31.880452: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-16 15:29:31.880839: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
df = pd.read_excel("../data/gbd1_data.xlsx", header=2, usecols="B:K,N:R")
f = open("../data/base_gbd1.txt")
base = f.readline()
f.close()

/home/surrogate/anaconda3/envs/tf-env/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
mutants, fitness = create_xy(df, base)

In [5]:
def aa_identity(aa):
    aa_string = "ARNDCQEGHILKMFPSTWYV"
    for i in range(len(aa_string)):
        if aa == aa_string[i]:
            return i

In [6]:
def m_identity(mt):
    mi = np.zeros((20, len(mt)))
    for i in range(len(mt)):
        row = aa_identity(mt[i])
        mi[row][i] = 1
    return mi

mutant_identity = []
for mutant in mutants:
    mutant_identity.append(m_identity(mutant))
mutant_identity = np.array(mutant_identity)
print(mutant_identity.shape)
print(fitness.shape)

(535917, 20, 56)
(535917, 2)


In [7]:
x_train, x_test, y_train, y_test = train_test_split(mutant_identity, fitness, test_size=0.2, shuffle=True, random_state=15)
print(f"{x_train.shape}\n{x_test.shape}\n{y_train.shape}\n{y_test.shape}")


(428733, 20, 56)
(107184, 20, 56)
(428733, 2)
(107184, 2)


In [11]:
mutant_identity = None
df = None
mutants = None
fitness = None

In [12]:
def model_1(x_train, y_train, x_test, y_test, epochs=20, batch_size=64):
    model = Sequential()
    model.add(keras.Input((20,56)))
    #model.add(Flatten())
    model.add(Conv1D(filters=128, kernel_size=12, activation='relu', use_bias=True, padding='same'))
    model.add(MaxPooling1D(2))    
    model.add(Conv1D(filters=64, kernel_size=6, activation='relu', use_bias=True))
    model.add(Conv1D(filters=32, kernel_size=4, activation='relu', use_bias=True))
    model.add(Flatten())
    model.add(Dense(128, activation='relu')) #evt tilføj dropout istedet for så mange dense layers
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='softmax'))  # Change to the appropriate number of classes for multi-class classification
    model.summary()
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(x_test, y_test))
    return model, history

In [13]:
model, history = model_1(x_train, y_train, x_test, y_test)

loss, accuracy = model.evaluate(x_test, y_test, batch_size=64, verbose=1)

print("Test Accuracy:", accuracy)

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

2024-07-16 15:02:10.763338: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-16 15:02:10.764983: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-16 15:02:10.765186: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 20, 128)        │        86,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 5, 64)          │        49,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 2, 32)          │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 162,306 (634.01 KB)

 Trainable params: 162,306 (634.01 KB)

...

Epoch 1/20


I0000 00:00:1721134937.289298   53630 service.cc:145] XLA service 0x71ae20002230 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721134937.289335   53630 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 6GB, Compute Capability 6.1
2024-07-16 15:02:17.372602: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-16 15:02:17.735441: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


  77/6699 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.5175 - loss: 0.5092 

I0000 00:00:1721134939.724636   53630 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6699/6699 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.7666 - loss: 0.3831 - val_accuracy: 0.8803 - val_loss: 0.3197
Epoch 2/20
6699/6699 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8828 - loss: 0.3225 - val_accuracy: 0.9108 - val_loss: 0.3136
Epoch 3/20
6699/6699 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9047 - loss: 0.3201 - val_accuracy: 0.8997 - val_loss: 0.3159
Epoch 4/20
6699/6699 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9086 - loss: 0.3155 - val_accuracy: 0.9351 - val_loss: 0.3093
Epoch 5/20
6699/6699 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9273 - loss: 0.3135 - val_accuracy: 0.9419 - val_loss: 0.3077
Epoch 6/20
6699/6699 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9317 - loss: 0.3125 - val_accuracy: 0.9143 - val_loss: 0.3112
Epoch 7/20
6699/6699 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9345 - loss: 0.3144 - val_accuracy: 0.9417 - val_loss: 0.3075
Epoch 8/20
6699/6699 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9381 - loss: 0.3108 - val